In [ ]:

import json

from adjustText import adjust_text

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rc
import torch

In [ ]:
embeddings = torch.load("../models/embeddings.ptx", map_location=torch.device('cpu'))

In [ ]:
embeddings

In [ ]:
def pairwise_similarity(matrix_of_vectors):
    '''
    Computes cosine similarities for between all vectors, extremely useful for comparing
    similarities between embeddings when doing deep embedding learning.

    Adapted from: https://github.com/dalisson/pairwise_cosine_distance_pytorch/blob/master/pairwise_cosine_similarity.py

    input:
        matrix_of_vectors: tensor with shape (n_vectors, vector_size)

    output:
        similarities : tensor with shape (n_vector, n_vectors)
    Each row[i, j] is the similarity of the ith element against the jth vector, eg,
    row[0,0] is 1 and row[0,42] is the similarity between the first
    element in the input and the 43th element in the input.
    '''

    dot_product = matrix_of_vectors@matrix_of_vectors.t()
    norms = torch.sqrt(torch.einsum('ii->i', dot_product))
    similarities = dot_product/(norms[None]*norms[..., None])

    return similarities

In [ ]:
similarities = pairwise_similarity(embeddings.weight) 

In [ ]:
with open('../models/MSFT-state.txt') as f:
    d = json.load(f)

In [ ]:
tsne_projections = pd.DataFrame(d[0]['projections'])
label = pd.read_csv('../models/metadata.tsv', sep='\t')

In [ ]:
label.tail()

In [ ]:
idx = label.index[label["label"] == "MSFT"].tolist()[0]
print(idx)

In [ ]:
idx_similiarities = similarities[idx].abs().tolist()

In [ ]:
params = {
    "pgf.texsystem": "xelatex",
    "pgf.rcfonts": False,
    "font.serif": [],
    "font.family": "serif",
    "font.sans-serif": [],
    "axes.labelsize": 11,
}

plt.rcParams.update(params)
rc("text", usetex=True)

plt.rc('text.latex', preamble=r'\usepackage{amsmath}\usepackage[utf8]{inputenc}')

CM = 1 / 2.54

In [ ]:
idx_distance = np.array([1- i for i in idx_similiarities])

In [ ]:
zorder = [int(o * 1000) for o in idx_similiarities]

In [ ]:
# filter for most similar underlyings
idx_labels = np.array(idx_similiarities) > 0.41

In [ ]:
tsne_projections.head()

In [ ]:
fig, ax = plt.subplots(figsize=(12 * CM, 8 * CM))

ax.scatter(tsne_projections['tsne-0'][~idx_labels], tsne_projections['tsne-1'][~idx_labels], c="whitesmoke", s=5) 

sc = ax.scatter(tsne_projections['tsne-0'][idx_labels], tsne_projections['tsne-1'][idx_labels], cmap='Blues_r', c=idx_distance[idx_labels], s=10, zorder=1000, marker="o", edgecolors="grey", linewidth=0.5)

ax.set_xlabel('$t$-SNE Axis 1')
ax.set_ylabel('$t$-SNE Axis 0')

texts = []

for i, cond in enumerate(idx_labels):
    if cond:
        l = label['label'].iloc[i]
        factor = 1.5 if l == 'MSFT' else 1

        texts.append(ax.text(tsne_projections['tsne-0'].iloc[i], tsne_projections['tsne-1'].iloc[i], r"\texttt{"+l+r"}",fontsize= 7 * factor, zorder=2000, ha="left", va="top"))

adjust_text(texts, ax=ax, min_arrow_len=2, arrowprops=dict(arrowstyle="-", color='k', lw=0.5))

fig.colorbar(sc)

fig.tight_layout()

plt.savefig('../reports/Graphs/categorical_embeddings.pdf', bbox_inches='tight')